## Initialization

In [6]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))
import torch
from kar_hnn import KARHNN
from data import get_dataset

## Help Functions

In [7]:
# print squared loss at specific steps for comparison with HNN
print_every = 200
def print_results(results, print_every=200):
    for step in range(0, len(results["train_loss"]), print_every):
        print(
            "step {}, train_loss {:.4e}, test_loss {:.4e}".format(
                step,
                results["train_loss"][step],
                results["test_loss"][step],
            )
        )
    print('Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}'
        .format(results["train_loss"][-1], results["train_std"][-1],
                results["test_loss"][-1], results["test_std"][-1]))

def print_best(results):
    curr_min = 0

    for step in range(0, len(results["train_loss"])):
        if results["test_loss"][step] < results["test_loss"][curr_min]:
            curr_min = step
    print(
        "best test loss at step {}, train_loss {:.4e}, test_loss {:.4e}".format(
            curr_min,
            results["train_loss"][curr_min],
            results["test_loss"][curr_min],
        )
    )


## Create Dataset

In [8]:
# create dataset
data = get_dataset(seed=0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(data["x"], requires_grad=True, dtype=torch.float32).to(device)
test_x = torch.tensor(data["test_x"], requires_grad=True, dtype=torch.float32).to(device)
dxdt = torch.Tensor(data["dx"]).to(device)
test_dxdt = torch.Tensor(data["test_dx"]).to(device)

# dataset['train_input'], dataset['train_label'],dataset['test_input'], dataset['test_label']
dataset = {
    "train_input": x,
    "train_label": dxdt,
    "test_input": test_x,
    "test_label": test_dxdt,
}

In [9]:
dataset["train_input"].shape, dataset["train_label"].shape

(torch.Size([1125, 2]), torch.Size([1125, 2]))

## Model Training

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# initialize KAN with G=3
model = KARHNN(input_dim=2, width=[2, 2, 2], grid=2, k=3, seed=1, device=device)
results = model.fit(dataset, opt="LBFGS", steps=53, log=1)
print_results(results, 1)
print_best(results)

cpu
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 3.28e-02 | test_loss: 3.43e-02 | reg: 1.20e+01 | : 100%|█| 53/53 [00:06<00:00,  8.47it

saving model version 0.1
step 0, train_loss 2.2177e-01, test_loss 2.4829e-01
step 1, train_loss 3.6794e-02, test_loss 3.8893e-02
step 2, train_loss 3.5242e-02, test_loss 3.9913e-02
step 3, train_loss 3.4616e-02, test_loss 4.0011e-02
step 4, train_loss 3.4519e-02, test_loss 3.8988e-02
step 5, train_loss 3.4693e-02, test_loss 3.8446e-02
step 6, train_loss 3.3669e-02, test_loss 3.5374e-02
step 7, train_loss 3.3462e-02, test_loss 3.5531e-02
step 8, train_loss 3.3360e-02, test_loss 3.5249e-02
step 9, train_loss 3.3224e-02, test_loss 3.5908e-02
step 10, train_loss 3.4637e-02, test_loss 3.8893e-02
step 11, train_loss 3.4555e-02, test_loss 3.8958e-02
step 12, train_loss 3.4308e-02, test_loss 3.7882e-02
step 13, train_loss 3.3789e-02, test_loss 3.5607e-02
step 14, train_loss 3.3391e-02, test_loss 3.5359e-02
step 15, train_loss 3.3695e-02, test_loss 3.6055e-02
step 16, train_loss 3.3408e-02, test_loss 3.4925e-02
step 17, train_loss 3.3240e-02, test_loss 3.4778e-02
step 18, train_loss 3.3118e-02,

![hnn](./hnn.png)
![baseline](./baseline.png)
